In [2]:
import pandas as pd
import numpy as np

import datetime

### Config

In [3]:
# DATA PATHS
TRAIN_PATH = 'data/data_training.csv'
PROV_PATH = 'data/data_provisional.csv'
SAMPLE_PATH = 'data/sample-data.csv'
# OUTPUT PATHS
OUTPUT_PATH = 'submissions/output/'

# Sample Data Paths
S_TRAIN = 'sample-submission/code/data/training.csv'
S_TEST = 'sample-submission/code/data/testing.csv'
S_SOL = 'sample-submission/solution/solution.csv'

category = {'low':0, 'medium':1, 'high':2, 'baseline':3, 'channelized':4, 'surprise':5}
rev_category = { v:k for k,v in category.items()}
output_header = ['timestamp', 'test_suite', 'predicted_induced_state',
       'three_sec_predicted_induced_state',
       'predicted_induced_state_confidence',
       'three_sec_predicted_induced_state_confidence', 'top_three_features']

In [4]:
sample_sub = pd.read_csv('sample-submission/solution/solution.csv')
sample_test = pd.read_csv('sample-submission/code/data/testing.csv')

In [ ]:
sample_test

In [15]:
df = sample_test.copy()
df['timestamp'] = pd.to_numeric(df['time'])
df['timestamp'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000000))
df['timestamp'] = df['timestamp'].dt.round('1s')
df['timestamp'] = df['timestamp'].apply(lambda x: int(datetime.datetime.timestamp(x)*1000000))
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]


In [13]:
df.head(2)

,time,test_suite,tlx_score,E4_BVP,E4_GSR,LooxidLink_EEG_A3,LooxidLink_EEG_A4,LooxidLink_EEG_FP1,LooxidLink_EEG_FP2,LooxidLink_EEG_A7,...,ViveEye_pupilPos_R_Y,ViveEye_gazeOrigin_R_X,ViveEye_gazeOrigin_R_Y,ViveEye_gazeOrigin_R_Z,ViveEye_gazeDirection_R_X,ViveEye_gazeDirection_R_Y,ViveEye_gazeDirection_R_Z,Zephyr_HR,Zephyr_HRV,timestamp
0,1626275615977130,Provisional-Test-025,10,17.03023,6.185013,-0.004482,-0.055402,0.031834,0.015929,0.076924,...,-1.0,-26.5375,-6.44585,-39.1862,0.0,0.0,0.0,105.0,20.0,1626275616000000
1,1626275616500321,Provisional-Test-025,10,-65.02724,6.195257,-0.004500,-0.055253,0.031860,0.015987,0.076948,...,-1.0,-27.4503,-6.37874,-39.1532,0.0,0.0,0.0,105.0,20.0,1626275617000000


In [14]:
df[cols].head(2)

,timestamp,time,test_suite,tlx_score,E4_BVP,E4_GSR,LooxidLink_EEG_A3,LooxidLink_EEG_A4,LooxidLink_EEG_FP1,LooxidLink_EEG_FP2,...,ViveEye_pupilPos_R_X,ViveEye_pupilPos_R_Y,ViveEye_gazeOrigin_R_X,ViveEye_gazeOrigin_R_Y,ViveEye_gazeOrigin_R_Z,ViveEye_gazeDirection_R_X,ViveEye_gazeDirection_R_Y,ViveEye_gazeDirection_R_Z,Zephyr_HR,Zephyr_HRV
0,1626275616000000,1626275615977130,Provisional-Test-025,10,17.03023,6.185013,-0.004482,-0.055402,0.031834,0.015929,...,-1.0,-1.0,-26.5375,-6.44585,-39.1862,0.0,0.0,0.0,105.0,20.0
1,1626275617000000,1626275616500321,Provisional-Test-025,10,-65.02724,6.195257,-0.004500,-0.055253,0.031860,0.015987,...,-1.0,-1.0,-27.4503,-6.37874,-39.1532,0.0,0.0,0.0,105.0,20.0


In [3]:
category = {'low':0, 'medium':1, 'high':2, 'baseline':3, 'channelized':4, 'surprise':5}
states = ['low', 'medium', 'high', 'baseline', 'channelized', 'surprise']
features = ['E4_BVP', 'E4_GSR', 'LooxidLink_EEG_A3', 'LooxidLink_EEG_A4', 'LooxidLink_EEG_FP1', 'LooxidLink_EEG_FP2',
                    'LooxidLink_EEG_A7', 'LooxidLink_EEG_A8', 'Muse_EEG_TP9', 'Muse_EEG_AF7', 'Muse_EEG_AF8', 'Muse_EEG_TP10',
                    'Muse_PPG_0', 'Muse_PPG_1', 'Muse_PPG_2', 'Myo_GYR_X', 'Myo_GYR_Y', 'Myo_GYR_Z', 'Myo_EMG_0', 'Myo_EMG_1',
                    'Myo_EMG_2', 'Myo_EMG_3', 'Myo_EMG_4', 'Myo_EMG_5', 'Myo_EMG_6', 'Myo_EMG_7', 'PICARD_fnirs_0', 'PICARD_fnirs_1',
                    'Polar_bpm', 'Polar_hrv', 'ViveEye_eyeOpenness_L', 'ViveEye_pupilDiameter_L', 'ViveEye_pupilPos_L_X',
                    'ViveEye_pupilPos_L_Y', 'ViveEye_gazeOrigin_L_X', 'ViveEye_gazeOrigin_L_Y', 'ViveEye_gazeOrigin_L_Z',
                    'ViveEye_gazeDirection_L_X', 'ViveEye_gazeDirection_L_Y', 'ViveEye_gazeDirection_L_Z', 'ViveEye_eyeOpenness_R',
                    'ViveEye_pupilDiameter_R', 'ViveEye_pupilPos_R_X', 'ViveEye_pupilPos_R_Y', 'ViveEye_gazeOrigin_R_X',
                    'ViveEye_gazeOrigin_R_Y', 'ViveEye_gazeOrigin_R_Z', 'ViveEye_gazeDirection_R_X', 'ViveEye_gazeDirection_R_Y',
                    'ViveEye_gazeDirection_R_Z', 'Zephyr_HR', 'Zephyr_HRV']


In [4]:
sub = submission.copy()

In [5]:
sub = sub.drop_duplicates()
sub = sub.reset_index(drop = True)

In [6]:
sub_final = ((sub.groupby(['timestamp','test_suite'])).max()).reset_index()

In [22]:
sub[~sub.eq(sub_final)].sum()

timestamp                                       0.0
test_suite                                        0
predicted_induced_state                           0
three_sec_predicted_induced_state                 0
predicted_induced_state_confidence                0
three_sec_predicted_induced_state_confidence      0
top_three_features                                0
dtype: object

In [26]:
round(2.6)

3

In [33]:
isinstance(['baseline'], list)

True

## Failed Submit

In [4]:
sub = pd.read_csv('submissions/draft_submission/solution/solution.csv')

In [9]:
sub.shape

(28267, 7)

In [5]:
sub.head()

,timestamp,test_suite,predicted_induced_state,three_sec_predicted_induced_state,predicted_induced_state_confidence,three_sec_predicted_induced_state_confidence,top_three_features
0,1626275615977130,Provisional-Test-025,baseline,baseline,"[0.01716378517448902, 0.0118869598954916, 0.01...","[0.01716378517448902, 0.0118869598954916, 0.01...",['Myo_EMG_2' 'LooxidLink_EEG_FP1' 'ViveEye_pup...
1,1626275616500321,Provisional-Test-025,baseline,baseline,"[0.015069910325109959, 0.011381322517991066, 0...","[0.015069910325109959, 0.011381322517991066, 0...",['Muse_EEG_TP10' 'Muse_EEG_TP9' 'LooxidLink_EE...
2,1626275617501578,Provisional-Test-025,baseline,baseline,"[0.017284145578742027, 0.011498662643134594, 0...","[0.017284145578742027, 0.011498662643134594, 0...",['ViveEye_gazeDirection_L_X' 'Myo_EMG_2' 'Vive...
3,1626275618500395,Provisional-Test-025,baseline,baseline,"[0.015703603625297546, 0.011379184201359749, 0...","[0.015703603625297546, 0.011379184201359749, 0...",['ViveEye_pupilPos_L_Y' 'ViveEye_eyeOpenness_R...
4,1626275619501968,Provisional-Test-025,baseline,baseline,"[0.01639132760465145, 0.011688385158777237, 0....","[0.01639132760465145, 0.011688385158777237, 0....",['ViveEye_pupilDiameter_R' 'ViveEye_pupilPos_R...


In [19]:
conf = sub.predicted_induced_state_confidence[0]

In [24]:
sub.predicted_induced_state_confidence.apply(lambda x: x if isinstance(x, list) else [float(y) for y in x.replace('[','').replace(']','').split()])

ValueError: could not convert string to float: '0.01716378517448902,'

In [25]:
sub.predicted_induced_state_confidence[0]

'[0.01716378517448902, 0.0118869598954916, 0.010458354838192463, 0.9356419444084167, 0.014097582548856735, 0.01075142901390791]'

In [26]:
sample_sub.predicted_induced_state_confidence[0]

'[0.283 0.609 0.656 0.391 0.897 0.863]'

In [23]:
sample_sub.predicted_induced_state_confidence.apply(lambda x: x if isinstance(x, list) else [float(y) for y in x.replace('[','').replace(']','').split()])

0        [0.283, 0.609, 0.656, 0.391, 0.897, 0.863]
1        [0.007, 0.485, 0.693, 0.735, 0.149, 0.004]
2        [0.505, 0.365, 0.195, 0.467, 0.496, 0.511]
3         [0.16, 0.235, 0.859, 0.074, 0.595, 0.924]
4          [0.687, 0.378, 0.819, 0.749, 0.87, 0.01]
                            ...                    
28262    [0.893, 0.548, 0.494, 0.424, 0.801, 0.963]
28263    [0.103, 0.252, 0.164, 0.462, 0.621, 0.912]
28264     [0.135, 0.285, 0.003, 0.559, 0.392, 0.16]
28265      [0.759, 0.18, 0.45, 0.287, 0.299, 0.291]
28266    [0.658, 0.549, 0.921, 0.948, 0.169, 0.311]
Name: predicted_induced_state_confidence, Length: 28267, dtype: object

In [7]:
sample_sub = pd.read_csv('sample-submission/solution/solution.csv')

In [8]:
sample_sub.head()

,timestamp,test_suite,predicted_induced_state,three_sec_predicted_induced_state,predicted_induced_state_confidence,three_sec_predicted_induced_state_confidence,top_three_features
0,1626275616000000,Provisional-Test-025,high,high,[0.283 0.609 0.656 0.391 0.897 0.863],[0.95 0.986 0.137 0.793 0.188 0.808],['Myo_EMG_2' 'LooxidLink_EEG_FP1' 'ViveEye_pup...
1,1626275617000000,Provisional-Test-025,high,high,[0.007 0.485 0.693 0.735 0.149 0.004],[0.556 0.07 0.68 0.286 0.721 0.821],['Muse_EEG_TP10' 'Muse_EEG_TP9' 'LooxidLink_EE...
2,1626275618000000,Provisional-Test-025,high,high,[0.505 0.365 0.195 0.467 0.496 0.511],[0.299 0.687 0.999 0.185 0.336 0.284],['ViveEye_gazeDirection_L_X' 'Myo_EMG_2' 'Vive...
3,1626275619000000,Provisional-Test-025,baseline,baseline,[0.16 0.235 0.859 0.074 0.595 0.924],[0.798 0.045 0.238 0.221 0.608 0.218],['ViveEye_pupilPos_L_Y' 'ViveEye_eyeOpenness_R...
4,1626275620000000,Provisional-Test-025,baseline,baseline,[0.687 0.378 0.819 0.749 0.87 0.01 ],[0.364 0.606 0.203 0.132 0.415 0.128],['ViveEye_pupilDiameter_R' 'ViveEye_pupilPos_R...


In [17]:
sample_sub.predicted_induced_state_confidence[0]

'[0.283 0.609 0.656 0.391 0.897 0.863]'

## 2nd failed submit

In [50]:
sample_data = pd.read_csv(SAMPLE_PATH)

In [10]:
s_sol = pd.read_csv(S_SOL)
s_test = pd.read_csv(S_TEST)
prev = pd.read_csv(OUTPUT_PATH+'solution_sample_1500.csv')
my = pd.read_csv(OUTPUT_PATH+'solution_train_prov_1500.csv')

In [56]:
sample_data.test_suite.isin(s_sol.test_suite)

0        False
1        False
2        False
3        False
4        False
         ...  
31118    False
31119    False
31120    False
31121    False
31122    False
Name: test_suite, Length: 31123, dtype: bool

In [5]:
s_sol.head()

,timestamp,test_suite,predicted_induced_state,three_sec_predicted_induced_state,predicted_induced_state_confidence,three_sec_predicted_induced_state_confidence,top_three_features
0,1626275616000000,Provisional-Test-025,high,high,[0.283 0.609 0.656 0.391 0.897 0.863],[0.95 0.986 0.137 0.793 0.188 0.808],['Myo_EMG_2' 'LooxidLink_EEG_FP1' 'ViveEye_pup...
1,1626275617000000,Provisional-Test-025,high,high,[0.007 0.485 0.693 0.735 0.149 0.004],[0.556 0.07 0.68 0.286 0.721 0.821],['Muse_EEG_TP10' 'Muse_EEG_TP9' 'LooxidLink_EE...
2,1626275618000000,Provisional-Test-025,high,high,[0.505 0.365 0.195 0.467 0.496 0.511],[0.299 0.687 0.999 0.185 0.336 0.284],['ViveEye_gazeDirection_L_X' 'Myo_EMG_2' 'Vive...
3,1626275619000000,Provisional-Test-025,baseline,baseline,[0.16 0.235 0.859 0.074 0.595 0.924],[0.798 0.045 0.238 0.221 0.608 0.218],['ViveEye_pupilPos_L_Y' 'ViveEye_eyeOpenness_R...
4,1626275620000000,Provisional-Test-025,baseline,baseline,[0.687 0.378 0.819 0.749 0.87 0.01 ],[0.364 0.606 0.203 0.132 0.415 0.128],['ViveEye_pupilDiameter_R' 'ViveEye_pupilPos_R...


In [20]:
my.shape

(28267, 7)

In [19]:
sum(my.three_sec_predicted_induced_state_confidence != prev.three_sec_predicted_induced_state_confidence)

28229

In [21]:
prev.shape

(28267, 7)

In [35]:
sum(s_sol.predicted_induced_state != my.predicted_induced_state)/s_sol.shape[0]

0.7200268864753954

In [36]:
sum(s_sol.predicted_induced_state_confidence != my.predicted_induced_state_confidence)

28267

In [17]:
type(s_sol.predicted_induced_state_confidence[0])

str

In [23]:
my['predicted_induced_state_confidence'] = my.predicted_induced_state_confidence.apply(lambda x: x.replace("\'", ""))
my['three_sec_predicted_induced_state_confidence'] = my.three_sec_predicted_induced_state_confidence.apply(lambda x: x.replace("\'", ""))

In [27]:
my.to_csv('solution_sample_800.csv', index=False)

### Checking if S_TEST & SAMPLE data are valid real data extracted from training

In [54]:
sample_data[sample_data.test_suite == 'Training-Test-038'].head()

,time,test_suite,induced_state,tlx_score,E4_BVP,E4_GSR,LooxidLink_EEG_A3,LooxidLink_EEG_A4,LooxidLink_EEG_FP1,LooxidLink_EEG_FP2,...,ViveEye_pupilPos_R_X,ViveEye_pupilPos_R_Y,ViveEye_gazeOrigin_R_X,ViveEye_gazeOrigin_R_Y,ViveEye_gazeOrigin_R_Z,ViveEye_gazeDirection_R_X,ViveEye_gazeDirection_R_Y,ViveEye_gazeDirection_R_Z,Zephyr_HR,Zephyr_HRV
0,1626275540354390,Training-Test-038,baseline,10,-9999.900000,-9999.900000,-0.002007,-0.059057,0.030180,0.010290,...,-9999.900000,-9999.900000,-9999.9000,-9999.90000,-9999.9000,-9999.900000,-9999.90000,-9999.900000,-9999.9,-9999.9
1,1626275540500599,Training-Test-038,baseline,10,-9999.900000,-9999.900000,-0.002007,-0.059079,0.030179,0.010289,...,-1.000000,-1.000000,-27.2645,-7.50266,-37.9508,0.000000,0.00000,0.000000,-9999.9,-9999.9
2,1626275541501825,Training-Test-038,baseline,10,-21.185060,7.358181,-0.002018,-0.059191,0.030173,0.010314,...,0.274257,0.938343,-27.1029,-7.38116,-37.9733,0.334305,0.16777,0.927399,102.0,26.0
3,1626275542501052,Training-Test-038,baseline,10,-3.672516,7.358181,-0.002375,-0.058885,0.030056,0.010255,...,-1.000000,-1.000000,-27.2440,-7.43414,-37.9404,0.000000,0.00000,0.000000,100.0,26.0
4,1626275543500284,Training-Test-038,baseline,10,1.859863,7.358181,-0.002526,-0.058668,0.030026,0.010284,...,-1.000000,-1.000000,-27.0874,-7.92627,-38.2178,0.000000,0.00000,0.000000,102.0,26.0
